In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from seaborn import set_style
set_style("whitegrid")

In [ ]:
df = pd.read_json("./clean_data.dat")
df.head()

In [ ]:
df = df.loc[df.loc[:, "Kategorija pokazatelja"].notna()]
df.loc[:, "SR.VR."] = df.loc[:, "SR.VR."].apply(lambda x: "".join(x.split(",")) if len(x.split()) <=2 else np.nan).apply(lambda x: np.nan if x == "" else x).apply(float)
df = df.loc[df.loc[:, "SR.VR."].notna()]

In [ ]:
df.info()

In [ ]:
df.loc[:, "SR.VR."]

In [ ]:
vodniTipovi = [item for item in list(df.loc[:, "Vodi tip"].unique()) if item != ""]

In [ ]:
kategorijePokazatelja = [item for item in list(df.loc[:, "Kategorija pokazatelja"].unique()) if item is not np.nan]

In [ ]:
kategorijePokazatelja

In [ ]:
pokazateljiPoKategorijama = {kategorija : [item for item in list(df.loc[df.loc[:, "Kategorija pokazatelja"] == kategorija, "Pokazatelj"].unique()) if item != ""] for kategorija in kategorijePokazatelja}

In [ ]:
pokazateljiPoKategorijama

Treba pivotat tablicu.<br>
Svaki pokazatalje će biti stupac, redak će predstavljati srednje vrijednosti identificirane (indeksirane) po lokaciji.

In [ ]:
df.pivot(index = ["Naziv postaje", "Razdoblje"], columns = "Pokazatelj", values = "SR.VR.")

In [ ]:
df.pivot_table(values = "SR.VR.", columns="Pokazatelj", index = "Naziv postaje")